In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]
GROQ_API_KEY = os.environ["GROQ_API_KEY"]

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/be-good.txt")

loaded_docs = loader.load()

In [4]:
loaded_docs[0].page_content

'Be good\n\nApril 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum that one\nor both of the principles we began with is false.  Or we have 

## Character Text Splitter

In [6]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000, 
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False
)

In [11]:
text = text_splitter.create_documents([loaded_docs[0].page_content])

In [12]:
text

[Document(metadata={}, page_content='Be good'),
 Document(metadata={}, page_content='April 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the\nphrase that became our motto: Make something people want.  We\'ve\nlearned a lot since then, but if I were choosing now that\'s still\nthe one I\'d pick.Another thing we tell founders is not to worry too much about the\nbusiness model, at least at first.  Not because making money is\nunimportant, but because it\'s so much easier than building something\ngreat.A couple weeks ago I realized that if you put those two ideas\ntogether, you get something surprising.  Make something people want.\nDon\'t worry too much about making money.  What you\'ve got is a\ndescription of a charity.When you get an unexpected result like this, it could either be a\nbug or a new discovery.  Either businesses aren\'t supposed to be\nlike charities, and we\'ve proven by reductio ad absurdum

## Recursive Character Text Splitter

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=26,
    chunk_overlap=4
)

text = recursive_text_splitter.split_text(loaded_docs[0].page_content)

In [16]:
text

['Be good',
 'April 2008(This essay is',
 'is derived from a talk at',
 'at the 2008 Startup',
 'School.)About a month',
 'after we started Y',
 'Y Combinator we came up',
 'up with the',
 'phrase that became our',
 'our motto: Make something',
 "people want.  We've",
 'learned a lot since then,',
 'but if I were choosing',
 "now that's still",
 "the one I'd pick.Another",
 'thing we tell founders is',
 'is not to worry too much',
 'about the',
 'business model, at least',
 'at first.  Not because',
 'making money is',
 'unimportant, but because',
 "it's so much easier than",
 'building something',
 'great.A couple weeks ago',
 'ago I realized that if',
 'if you put those two',
 'two ideas',
 'together, you get',
 'get something surprising.',
 'Make something people',
 'want.',
 "Don't worry too much",
 'about making money.  What',
 "you've got is a",
 'description of a',
 'a charity.When you get an',
 'an unexpected result like',
 'this, it could either be',
 'be a',
 'bug or a new di

## Embeddings

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001",google_api_key=GEMINI_API_KEY)



In [20]:
chunks = [
        "Today is Monday",
        "Today is Tuesday",
        "Today is April Fools day",
    ]

In [23]:
embeddings = embeddings.embed_documents(chunks)
embeddings

[[-0.03623005375266075,
  0.0030203552450984716,
  0.007686538156121969,
  -0.09647919982671738,
  -0.008500777184963226,
  0.01038917526602745,
  -0.0010574186453595757,
  -0.005027700215578079,
  -0.0009292842005379498,
  -0.0048210835084319115,
  -0.009886398911476135,
  -0.018536018207669258,
  0.0031254631467163563,
  0.01243983767926693,
  0.15042933821678162,
  -0.007787199690937996,
  -0.0068469904363155365,
  -0.000732616928871721,
  -0.0028274066280573606,
  -0.00848695170134306,
  -0.002090933732688427,
  -0.0046785688027739525,
  0.021853847429156303,
  -0.015931101515889168,
  -0.00254137278534472,
  0.010295355692505836,
  -0.010196185670793056,
  -0.0018972809193655849,
  0.04515213891863823,
  -0.0010483571095392108,
  -0.0013804897898808122,
  0.020583167672157288,
  -0.005922070238739252,
  0.00656348979100585,
  0.01237465813755989,
  -0.006826752796769142,
  0.020590320229530334,
  0.003684631083160639,
  0.0020155254751443863,
  -9.710732228995766e-06,
  0.00574388

In [25]:
len(embeddings[0])

3072

## Vector DB

In [37]:
from langchain_community.document_loaders import TextLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma

loaded_document = TextLoader('data/state_of_the_union.txt').load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

chunks_of_text = text_splitter.split_documents(loaded_document)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)

vector_db = Chroma.from_documents(chunks_of_text, embeddings)

In [40]:
question = "What did the president say about Ketanji Brown Jackson?"

response = vector_db.similarity_search(question)

print(response[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## vector store as retriever

In [43]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/state_of_the_union.txt")

In [46]:
from langchain_community.document_loaders import TextLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

loaded_document = TextLoader('data/state_of_the_union.txt').load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

chunks_of_text = text_splitter.split_documents(loaded_document)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)

vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [48]:
retriver =  vector_db.as_retriever()

In [49]:
response = retriver.invoke("what did he say about Ketanji Brown Jackson?")
response

[Document(id='8b8d2190-038a-44a8-9565-4566e5af05f2', metadata={'source': 'data/state_of_the_union.txt'}, page_content='And my report is this: the State of the Union is strong—because you, the American people, are strong. \n\nWe are stronger today than we were a year ago. \n\nAnd we will be stronger a year from now than we are today. \n\nNow is our moment to meet and overcome the challenges of our time. \n\nAnd we will, as one people. \n\nOne America. \n\nThe United States of America. \n\nMay God bless you all. May God protect our troops.'),
 Document(id='38f934a2-c4bd-4047-8327-a2f49fa82173', metadata={'source': 'data/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Cons